In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad", split="validation")  

contexts = dataset["context"]
questions = dataset["question"]
answers = dataset["answers"]

In [ ]:
from langchain.schema import Document 
import json

contexts_with_qa: Dict[str, List[Dict]] = {}
for i, ex in tqdm(enumerate(dataset), total=len(ds)):
        ctx = ex["context"]
        ans_txt = ex["answers"]["text"][0] if ex["answers"]["text"] else ""
        ans_st = ex["answers"]["answer_start"][0] if ex["answers"]["answer_start"] else ""
        qid = ex.get("id", str(i))
        qa_pair = {
        "question":ex.get("question", ""),
        "answer_text": ans_txt,
        "answer_start": ans_st,
        "id": qid
        }

        contexts_with_qa.setdefault(ctx, []).append(qa_pair)

texts = []
metadatas = []
for ctx, qa_list in contexts_with_qa.items():
        texts.append(ctx)
        metadatas.append({"qas": json.dumps(qa_list)})

docs = [Document(page_content=text, metadata=meta)
        for text, meta in zip(texts, metadatas)]



In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_model)
retriever = db.as_retriever()

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")
llm = HuggingFacePipeline(pipeline=qa_pipeline)

In [ ]:
from langchain.memory import ConversationBufferMemory

# 1. Initialize memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 2. (Re)define retriever and qa_pipeline if needed—
#    otherwise skip these two lines
# from transformers import pipeline
# qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# 3. Your original retrieval + QA code, now with memory
query = "Tell me about Jay Z and Beyonce"

# Step 3a. Retrieve relevant docs
docs = retriever.get_relevant_documents(query)


# Step 3b. Build docs context
k = 2  # Number of relevant docs to retrieve
docs_context = "\n\n".join([d.page_content for d in docs[:k]])

# Step 3c. Pull history from memory
history_msgs = memory.chat_memory.messages  # list of Message objects
history_str = "\n".join(
    f"{msg.type.capitalize()}: {msg.content}" for msg in history_msgs
)

# Step 3d. Combine history + docs
full_context = (history_str + "\n\n" + docs_context) if history_str else docs_context

print(full_context)

# Step 4. Call the QA pipeline
result = qa_pipeline(question=query, context=full_context)

print("Answer:", result["answer"])

# Step 5. Store this turn in memory
memory.chat_memory.add_user_message(query)
memory.chat_memory.add_ai_message(result["answer"])